 # asyncio使用

### asycio简介

asyncio是Python 3.4版本引入的标准库，直接内置了对异步IO的支持。

asyncio的编程模型就是一个消息循环。我们从asyncio模块中直接获取一个EventLoop的引用，然后把需要执行的协程扔到EventLoop中执行，就实现了异步IO。

### asyncio 是干什么的？

* 异步网络操作
* 并发
* 协程

现在的asyncio，有了很多的模块已经在支持：aiohttp,aiodns,aioredis等等 ,

(https://github.com/aio-libs)[https://github.com/aio-libs] 这里列出了已经支持的内容，并在持续更新

当然到目前为止实现协程的不仅仅只有asyncio,tornado和gevent都实现了类似功能.

### 关于asyncio的一些关键字的说明

* event_loop 事件循环：程序开启一个无限循环，把一些函数注册到事件循环上，当满足事件发生的时候，调用相应的协程函数

* coroutine 协程：协程对象，指一个使用async关键字定义的函数，它的调用不会立即执行函数，而是会返回一个协程对象。协程对象需要注册到事件循环，由事件循环调用。

* task 任务：一个协程对象就是一个原生可以挂起的函数，任务则是对协程进一步封装，其中包含了任务的各种状态.

* future: 代表将来执行或没有执行的任务的结果。它和task上没有本质上的区别.

* async/await 关键字：python3.5用于定义协程的关键字
    * async定义一个协程
    * await用于挂起阻塞的异步调用接口。
    
注意: 

1. 通过async关键字定义一个协程（coroutine）,协程是接运行，必须将协程加入到事件循环loop中运行.
2. asyncio.get_event_loop：创建一个事件循环，然后使用run_until_complete将协程注册到事件循环，并启动事件循环

## asycio使用

### asycio执行一个任务

In [ ]:
import asyncio

@asyncio.coroutine
def hello():
    print("Hello world!")
    # 异步调用asyncio.sleep(1):
    r = yield from asyncio.sleep(1)
    print("Hello again!")

# 获取EventLoop:
loop = asyncio.get_event_loop()
# 执行coroutine, 注意是:hello(),不是hello
loop.run_until_complete(hello())
loop.close()

In [2]:
# 使用3.5版本async和await关键字
import asyncio

async def hello():
    print('hello,world')
    r=await asyncio.sleep(1)
    print('hello,again')

loop=asyncio.get_event_loop()
loop.run_until_complete(hello())
loop.close()

注意:
* loop.run_until_complete(hello())的参数是一个协程对象,不是函数的名字
* yield from 在3.7版本要废弃掉,最好使用async和await


### asycio执行多个任务

In [ ]:
import threading
import asyncio

@asyncio.coroutine
def hello():
    print('Hello world! (%s)' % threading.currentThread())
    yield from asyncio.sleep(1)
    print('Hello again! (%s)' % threading.currentThread())

loop = asyncio.get_event_loop()
tasks = [hello(), hello()]
loop.run_until_complete(asyncio.wait(tasks))
loop.close()

### 使用asycio异步网络连接来获取sina、sohu和163的网站首页,并打印header内容

In [ ]:
import asyncio

@asyncio.coroutine
def wget(host):
    print('wget %s...' % host)
    connect = asyncio.open_connection(host, 80)
    reader, writer = yield from connect
    header = 'GET / HTTP/1.0\r\nHost: %s\r\n\r\n' % host
    writer.write(header.encode('utf-8'))
    yield from writer.drain()
    while True:
        line = yield from reader.readline()
        if line == b'\r\n':
            break
        print('%s header > %s' % (host, line.decode('utf-8').rstrip()))
    # Ignore the body, close the socket
    writer.close()

loop = asyncio.get_event_loop()
tasks = [wget(host) for host in ['www.sina.com.cn', 'www.sohu.com', 'www.163.com']]
loop.run_until_complete(asyncio.wait(tasks))
loop.close()

### 在一段Python程序中使用多次事件循环

在Python异步程序编写中经常要用到如下的结构:

```python
import asyncio
async def doAsync():
    await asyncio.sleep(1)
    #...
    
if __name__ == "__main__":
    loop = asyncio.get_event_loop()
    loop.run_until_complete(doAsync())
    loop.close()
``` 

这当然是很不错的,也是官方的用法，但当你第二次使用loop的时候程序就会抛出异常RuntimeError: Event loop is closed,这也无可厚非，理想的程序也应该是在一个时间循环中解决掉各种异步IO的问题。

在定时任务的场景下, 如果你想每次启动时间循环执行一些任务后关闭事件循环,等下一次执行任务时再重新启动事件循环,循环往复该怎么办呢?

解决方案:

我们可以使用`asyncio.new_event_loop`函数建立一个新的事件循环，并使用`asyncio.set_event_loop`设置为全局的事件循环，这时候就可以多次运行异步的事件循环了，不过最好保存默认的asyncio.get_event_loop并在事件循环结束的时候还原回去。

```python
import asyncio

async def doAsync():
    await asyncio.sleep(0)
    #...
    
def runEventLoop()
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    loop.run_until_complete(doAsync())
    loop.close()
    
if __name__ == "__main__":
    oldloop = asyncio.get_event_loop()
    for i in range(10):
        runEventLoop()
    asyncio.set_event_loop(oldloop)
```    